In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)#测试集比例占25%

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
#Evaluate Metrics 评价指标
accuracy.rmse(predictions)# Then compute RMSE
accuracy.mae(predictions)#Then compute NAE

RMSE: 0.9323
MAE:  0.7344


0.7344053841770187

In [1]:
from surprise import KNNBasic
from surprise import Dataset
data = Dataset.load_builtin('ml-100k') #加载movielens-100k数据集
trainset = data.build_full_trainset() #纠正/取出训练集
#sim_options={nam=cosine, user_based=True,min_support=10,shrinkage}
algo = KNNBasic(k=40, min_k=1,sim_options ={'name':'cosine','user_based': True}, verbose=True) #建立UCF算法并训练
algo.fit(trainset)
uid = str(196)  # 原始user id (在评分文件中的). 注意，是个字符串
iid = str(302)  # 原始item id （其他同上）
pred = algo.predict(uid, iid, r_ui=4, verbose=True) #对某一个具体的user和item给出预测

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 196        item: 302        r_ui = 4.00   est = 4.12   {'actual_k': 40, 'was_impossible': False}


In [2]:
from surprise import KNNBasic
from surprise import Dataset
data = Dataset.load_builtin('ml-100k') #加载movielens-100k数据集
trainset = data.build_full_trainset() #纠正/取出训练集
#sim_options={nam=cosine, user_based=True,min_support=10,shrinkage}
algo = KNNBasic(k=40, min_k=1,sim_options ={'name':'cosine','user_based': False}, verbose=True) #建立ICF算法并训练
algo.fit(trainset)
uid = str(196)  # 原始user id (在评分文件中的). 注意，是个字符串
iid = str(302)  # 原始item id （其他同上）
pred = algo.predict(uid, iid, r_ui=4, verbose=True) #对某一个具体的user和item给出预测

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 196        item: 302        r_ui = 4.00   est = 3.61   {'actual_k': 39, 'was_impossible': False}


In [3]:
from collections import defaultdict
from surprise import SVD
from surprise import Dataset
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n
# First train an UCF algorithm on the movielens dataset.
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()#纠正/取出训练集
algo = KNNBasic(k=40, min_k=1,sim_options ={'name':'cosine','user_based': True}, verbose=True) #建立UCF算法并训练
algo.fit(trainset)
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)
top_n = get_top_n(predictions, n=10)
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Computing the cosine similarity matrix...
Done computing similarity matrix.
196 ['1189', '1500', '814', '1536', '1293', '1599', '1653', '1467', '1122', '1201']
186 ['1189', '814', '1536', '1599', '1653', '1467', '1201', '1500', '1293', '1122']
22 ['1189', '1500', '814', '1536', '1599', '1656', '1653', '1467', '1122', '1201']
244 ['1189', '814', '1293', '1599', '1653', '1201', '1500', '1536', '1122', '483']
166 ['1189', '1500', '814', '1536', '1293', '1599', '1653', '1122', '1201', '1467']
298 ['1189', '814', '1536', '1293', '1599', '1653', '1467', '1122', '1201', '1500']
115 ['814', '1293', '1599', '1653', '1467', '1122', '1201', '1189', '1500', '1536']
253 ['1189', '1500', '814', '1536', '1293', '1599', '1653', '1467', '1122', '1201']
305 ['1500', '814', '1536', '1293', '1599', '1653', '1467', '1122', '1201', '1189']
6 ['1500', '814', '1536', '1293', '1599', '1467', '1122', '1201', '1189', '1653']
62 ['1189', '1500', '814', '1536', '1599', '1653', '1467', '1122', '1293', '1201']
286 [